In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import numpy as np
import pandas as pd
from pypfopt.expected_returns import ema_historical_return

In [3]:
from data import *

In [4]:
class dataset:
    data_path = '../data/SP500/SP_20180402_20200401.csv'

class train:
    start_date = '2019-04-01'
    end_date = '2019-08-01'

In [5]:
train_dataset = SP500_loader(dataset, train)

In [6]:
mu = ema_historical_return(train_dataset)
C = train_dataset.cov()

In [7]:
N = len(train_dataset.columns)
K = 30

In [8]:
randoms = np.random.uniform(0, 1, (K))
randoms = list(randoms / randoms.sum())
w = np.random.permutation([*randoms, *[0 for i in range(N - K)]])
ws = w.copy()

gamma = 0.95
gamma_u = 0.95
u = 0.1
T = 100
I = int(N * (N - 1) / 2)

In [10]:
for k in range(2 * N):
    wu = w.copy()
    choices = np.where(w != 0.0)[0]
    i = np.random.choice(choices)
    j = np.random.randint(N)
    if w[j] == 0:
        wu[j] = w[i]
        wu[i] = 0
    else:
        ui = ru = np.random.uniform(0, min(u, w[i], w[j]))
        wu[j] = w[j] + ui
        wu[i] = w[i] - ui
    dl = L(wu, mu, C) - L(w, mu, C)
    if dl > 0:
        w = wu.copy()
        if L(w, mu, C) > L(ws, mu, C):
            ws = w.copy()
    else:
        p = np.random.uniform(0, 1)
        if p <= np.exp(dl / T):
            w = wu.copy()
    T = gamma * T
    u = gamma_u * u

In [11]:
ws

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.0133049 , 0.        ,
       0.00625893, 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03665734, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.01290469, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.05803392,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.05428702, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.017041  , 0.        , 0.     

In [13]:
from utils.financial_measures import *

In [17]:
weights, \
corr_min, \
corr_max, \
corr_mean, \
corr_std, \
port_return, \
port_std, \
port_sharpe, \
port_information, \
port_modigliani = calculate_measures(train_dataset.columns, train_dataset, ws)

In [20]:
port_sharpe

2.76711415900063

Index(['AKAM', 'ALGN', 'AMD', 'ARNC', 'AYI', 'BLL', 'CAG', 'CERN', 'CF', 'DRE',
       'F', 'FE', 'FITB', 'GWW', 'HES', 'INFO', 'IR', 'KMX', 'MAT', 'O', 'PHM',
       'RCL', 'REGN', 'SBUX', 'SO', 'SPG', 'UAL', 'USB', 'WEC', 'ZBH'],
      dtype='object', name='name')